In [1]:
import os
from utils import setup_openai_key
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [2]:
# Save OpenAI key in the environment
setup_openai_key()

- Khi bạn đặt nó càng cao, bạn sẽ nhận được nhiều kết quả đầu ra ngẫu nhiên hơn.
- Khi bạn đặt nó càng thấp, về phía 0, các giá trị sẽ mang tính quyết định hơn.

[<img src="images/temperature_parameter.png" alt="Temperature Parameter" width="30%" height="30%">](https://www.coltsteele.com/tips/understanding-openai-s-temperature-parameter)

/)



In [3]:
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [4]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Jog Intervals: Start out with an easy 10 minute jog. Once warm, run for two minutes at a moderate intensity, followed by two minutes of jogging at a lighter intensity. Each interval should be repeated for a total of 25 minutes.

2. Hills: Find a safe, local hill and walk up it at a moderate pace for two minutes. Walk back down and repeat for a total of 15-20 minutes.

3. Swim: Find a local pool and swim laps. Aim for a total of 25 minutes of swimming, focusing on maintaining good form and improving stroke technique.

4. Step Climbing: Find a safe, local hill and use it to perform step climbing for 15 minutes. Keep the repetition going and make sure to switch up the speed and intensity every few minutes.

5. Stationary Bike: Find a bike and ride for 30-45 minutes at a light-moderate intensity. Make sure to take periodic breaks and vary the speed throughout the workout.

6. Adventure Run: Find a local trail or park and embark on a 25-30 minute adventure run. Make sure to take in the

In [5]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
print(chain.run("eco-friendly water bottles"))



Eco-Bottle Solutions.


In [8]:
# Memory

In [9]:
llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [10]:
conversation.predict(input="Tell me about yourself.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


" Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!"

In [11]:
conversation.predict(input="What can you do?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can you do?
AI:

> Finished chain.


" I can help you with a variety of tasks, such as scheduling appointments, setting reminders, and providing information. I'm also able to answer questions about topics like current events, sports, and entertainment. I'm always learning new things, so I'm sure I can help you with whatever you need."

In [ ]:
conversation.predict(input="How can you help me with data analysis?")


In [12]:
# Display the conversation
print(conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Tell me about yourself.', additional_kwargs={}, example=False), AIMessage(content=" Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!", additional_kwargs={}, example=False), HumanMessage(content='What can you do?', additional_kwargs={}, example=False), AIMessage(content=" I can help you with a variety of tasks, such as scheduling appointments, setting reminders, and providing information. I'm also able to answer questions about topics like current events, sports, and entertainment. I'm always learning new things, so I'm sure I can help you with whatever you need.", additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_pr

In [ ]:
# Deep Lake VectorStore

In [13]:
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [15]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [17]:
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "gioinguyen1305" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"

In [18]:
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Deep Lake Dataset in hub://gioinguyen1305/langchain_course_from_zero_to_hero already exists, loading from the storage


In [19]:
db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.44s/it]

Dataset(path='hub://gioinguyen1305/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['f7effdba-9d8b-11ee-b60b-04d9f5cc5714',
 'f7effdbb-9d8b-11ee-98a7-04d9f5cc5714']

In [20]:
# Let's create a RetrievalQA chain
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [21]:
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

In [22]:
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [25]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [26]:
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.72s/it]

Dataset(path='hub://gioinguyen1305/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
   text       text      (6, 1)      str     None   


['8fb7da2e-9d8c-11ee-b082-04d9f5cc5714',
 '8fb7da2f-9d8c-11ee-b8f9-04d9f5cc5714']

In [28]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


### Agents in LangChain

In [29]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [30]:
llm = OpenAI(model="text-davinci-003", temperature=0)

In [33]:
tools = [
    Tool(
        name = "google-search",
        func=retrieval_qa.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [34]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [35]:
response = agent("What's the latest news about the Mars rover?")
print(response['output'])



> Entering new  chain...
 I need to find out the latest news about the Mars rover
Action: google-search
Action Input: "latest news Mars rover"
Observation:  I don't know.
Thought: I should narrow my search
Action: google-search
Action Input: "latest news Mars rover 2020"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l


Observation:  The latest news about the Mars rover 2020 is that it has successfully landed on Mars.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l

 I now know the final answer
Final Answer: The latest news about the Mars rover 2020 is that it has successfully landed on Mars.

> Finished chain.
The latest news about the Mars rover 2020 is that it has successfully landed on Mars.


### Tools in LangChain

In [36]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [37]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [38]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [39]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find out the latest news about the Mars rover.
Action: google-search
Action Input: "latest news Mars rover"
Observation:  I don't know.
Thought: I should narrow my search to get more specific results.
Action: google-search
Action Input: "latest news Mars rover 2020"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l


Observation:  The latest news about the Mars rover 2020 is that it has successfully landed on Mars.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-d5K6E604XST4O6A5lxEPKB2e on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l

 I now know the final answer.
Final Answer: The latest news about the Mars rover 2020 is that it has successfully landed on Mars.

> Finished chain.
The latest news about the Mars rover 2020 is that it has successfully landed on Mars.
